In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["datasource", "dataproducts", "analyticsproducts"]
adls_paths = {tier: f"abfss://{tier}@nishantdev.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
datasource_adls = adls_paths["datasource"]
dataproducts_adls = adls_paths["dataproducts"]
analyticsproducts_adls = adls_paths["analyticsproducts"] 

dbutils.fs.ls(datasource_adls)
dbutils.fs.ls(dataproducts_adls)
dbutils.fs.ls(analyticsproducts_adls)

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()
     

In [0]:

start_date, end_date

In [0]:
#Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{datasource_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

In [0]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "datasource_adls": datasource_adls,
    "dataproducts_adls": dataproducts_adls,
    "analyticsproducts_adls": analyticsproducts_adls
}

# Return the dictionary directly
dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)